datasets used
* https://data.world/uci/arrhythmia/workspace/file?filename=arrhythmia.names.txt

here the same feature is used to build the while tree

* https://medium.com/@penggongting/implementing-decision-tree-from-scratch-in-python-c732e7c69aea
* https://philippmuens.com/decision-trees-from-scratch/

In [67]:
from sklearn.datasets import load_diabetes
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier

In [68]:
df = pd.read_csv('data/arrhythmia.data.csv', header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
0,75,0,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,8
1,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,6
2,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,10
3,55,0,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1
4,75,0,190,80,88,181,360,177,103,-16,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,7


In [62]:
%%time

class Node:
    def __init__(self, gini_indx, feature, midval, indices=None):
        self.gini_indx = gini_indx
        self.indices = indices
        self.midval = midval
        self.feature = feature
        self.left = None
        self.right = None
        
def select_best_gini(gini, bestgini, feature, best_feature, mean, bestmean):
    # for a perfectly pure node the gini impurity is 0
    bestgini = min(bestgini, gini)
    if bestgini == gini:
        best_feature = feature
        bestmean = mean
    return bestgini, best_feature, bestmean

def subnode_gini_impurity(df, indices):
    # reference https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    total_rows = len(indices)
    labels = df[279].unique()
    # 279 is the label node. so calculating the gini of the label
    gini = sum((len(df[df[279]==label]) / total_rows) ** 2 for label in labels)
    gini_impurity = 1 - gini
    return gini_impurity

class DecisionTree:
    def __init__(self, root, df):
        self.root = root
        self.df = df
        self.total_rows = len(df)
        
    def select_best_feature(self, df):
        best_gini = 10
        best_feature = None
        gini = 10
        bestmean = None
        features = df.columns[:-1]
        for feature in features:
            if df[feature].dtype == 'int64':
                distinct_mean = df[feature].unique().mean()
                if len(df[df[feature] == distinct_mean]) == len(df):
                    gini = subnode_gini_impurity(df, df.index.tolist())
                    
                else:
                    left_indices = df[df[feature] <= distinct_mean].index.tolist()
                    leftgini = subnode_gini_impurity(df.loc[left_indices], left_indices)
                    right_indices = df[df[feature] > distinct_mean].index.tolist()
                    rightgini = subnode_gini_impurity(df.loc[right_indices], right_indices)
                    gini = (len(left_indices)/len(df)) * leftgini + (len(right_indices)/len(df)) * rightgini
                    
            best_gini, best_feature, bestmean = select_best_gini(
                gini, best_gini, feature, best_feature, distinct_mean, bestmean)
        return best_gini, best_feature, bestmean
        
    def build_using_dfs(self, df, indices):
        """This is preorder traversal"""
#         print(len(indices))
        if not indices:
            return None
        
        thisdf = df.loc[indices]
        gini, feature, distinct_mean = self.select_best_feature(self, thisdf)
#         print(gini, feature, distinct_mean, len(indices))
        root = Node(gini, feature, distinct_mean, indices)
        if gini == 0:
            # this is a pure node. hence we dont branch out anymore
            distinct_mean = thisdf[feature].unique().mean()
            root = Node(0, feature, distinct_mean, indices)
            return root
        left_indices = thisdf[thisdf[feature] <= distinct_mean].index.tolist()
        right_indices = thisdf[thisdf[feature] > distinct_mean].index.tolist()
        root.left = self.build_using_dfs(self, df, left_indices)
        root.right = self.build_using_dfs(self, df, right_indices)
        return root
        
    @classmethod
    def build_from(cls, df):
        root = cls.build_using_dfs(cls, df, df.index.to_list())
        return cls(root, df)
    
    def search(self, testcase, node):
        if node:
            if node.gini_indx == 0:
                return self.df[279][node.indices[0]]
            midval = node.midval
            feature = node.feature
            testval = testcase[feature]
            if testval <= midval:
                return self.search(testcase, node.left) or self.df[279][node.indices[0]]
            else:
                return self.search(testcase, node.right) or self.df[279][node.indices[0]]
    
    def predict(self, testcase):
        node = self.root
        return self.search(testcase, node)
    
decision_tree = DecisionTree.build_from(df)

test_case = df.loc[0].to_dict()
print(decision_tree.predict(testcase))

NameError: name 'testcase' is not defined

In [63]:
test_case = df.loc[0].to_dict()
print(decision_tree.predict(test_case))

8


In [64]:
test_case

{0: 75,
 1: 0,
 2: 190,
 3: 80,
 4: 91,
 5: 193,
 6: 371,
 7: 174,
 8: 121,
 9: -16,
 10: '13',
 11: '64',
 12: '-2',
 13: '?',
 14: '63',
 15: 0,
 16: 52,
 17: 44,
 18: 0,
 19: 0,
 20: 32,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 44,
 29: 20,
 30: 36,
 31: 0,
 32: 28,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 52,
 40: 40,
 41: 0,
 42: 0,
 43: 0,
 44: 60,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 52,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 0,
 62: 0,
 63: 0,
 64: 56,
 65: 36,
 66: 0,
 67: 0,
 68: 32,
 69: 0,
 70: 0,
 71: 0,
 72: 0,
 73: 0,
 74: 0,
 75: 48,
 76: 32,
 77: 0,
 78: 0,
 79: 0,
 80: 56,
 81: 0,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 0,
 87: 80,
 88: 0,
 89: 0,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 40,
 101: 52,
 102: 0,
 103: 0,
 104: 28,
 105: 0,
 106: 0,
 107: 0,
 108: 0,
 109: 0,
 110: 0,
 111: 0,
 112: 48,
 113: 48,
 114: 0,
 115: 0,
 116: 32,
 117

In [45]:
def subnode_gini_impurity(df, indices):
    # reference https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
#     import pdb; pdb.set_trace()
    total_rows = len(indices)
    labels = df[279].unique()
    # 279 is the label node. so calculating the gini of the label
    gini = sum((len(df[df[279]==label]) / total_rows) ** 2 for label in labels)
    gini_impurity = 1 - gini
#     if gini > 1:
#         import pdb; pdb.set_trace()
    return gini_impurity

def select_best_feature(df):
    best_gini = 10
    best_feature = None
    gini = 10
    bestmean = None
    features = df.columns[:-1]
#     import pdb; pdb.set_trace()
    for feature in features:
        if df[feature].dtype == 'int64':
            distinct_mean = df[feature].unique().mean()
            if len(df[df[feature] == distinct_mean]) == len(df):
                gini = subnode_gini_impurity(df, df.index.tolist())
            else:
                left_indices = df[df[feature] <= distinct_mean].index.tolist()
                leftgini = subnode_gini_impurity(df.loc[left_indices], left_indices)
                right_indices = df[df[feature] > distinct_mean].index.tolist()
                rightgini = subnode_gini_impurity(df.loc[right_indices], right_indices)
                gini = (len(left_indices)/len(df)) * leftgini + (len(right_indices)/len(df)) * rightgini

        print(feature, gini)
        best_gini, best_feature, bestmean = select_best_gini(
            gini, best_gini, feature, best_feature, distinct_mean, bestmean)
    return best_gini, best_feature, bestmean



indices = df.index.to_list()

thisdf = df.loc[indices]
print(thisdf.shape)
gini, feature, distinct_mean = select_best_feature(thisdf)
print(gini, feature, distinct_mean)
# root = Node(feature, distinct_mean, indices)
# if len(thisdf[thisdf[feature] == distinct_mean]) == len(thisdf):
#     return root
# left_indices = thisdf[thisdf[feature] <= distinct_mean].index.tolist()
# right_indices = thisdf[thisdf[feature] > distinct_mean].index.tolist()
# root.left = self.build_using_dfs(self, df, left_indices)
# root.right = self.build_using_dfs(self, df, right_indices)

(452, 280)
0 0.6705104718806889
1 0.6597137930509255
2 0.6743248435854642
3 0.6730260229562779
4 0.6469469026548673
5 0.6740149599662874
6 0.6717362541538394
7 0.6516916934355717
8 0.6744978296366839
9 0.6727098921984795
10 0.6727098921984795
11 0.6727098921984795
12 0.6727098921984795
13 0.6727098921984795
14 0.6727098921984795
15 0.6703242192150483
16 0.6704552753936843
17 0.6520073179033358
18 0.6743264503441492
19 0.6753661210744772
20 0.6634718067091129
21 0.6728607028628614
22 0.6733721367622894
23 0.6744847656946011
24 0.6726253687315633
25 0.6719370698131759
26 0.6699391592920354
27 0.6599879845824643
28 0.6711387895666918
29 0.6583654161728955
30 0.6733369049036338
31 0.6729195690991505
32 0.6688034518003623
33 0.6747438810491908
34 0.669830250383398
35 0.6728607028628614
36 0.6741986234021631
37 0.6745427728613568
38 0.67216153638546
39 0.6708727124323396
40 0.6743948464341488
41 0.6718685208596713
42 0.672958686041517
43 0.6742969518190757
44 0.6685853392471321
45 0.67483272

### we can prove that the gini index can never be greater than 1

In [ ]:
indices = [5, 6, 13, 20, 22, 24, 26, 27, 37, 40, 41, 42, 43, 44, 47, 48, 49, 51, 52, 53, 56, 57, 59, 60, 61, 62, 63, 67, 70, 71, 72, 74, 75, 77, 80, 86, 87, 94, 96, 97, 99, 101, 102, 103, 104, 108, 111, 112, 113, 114, 115, 117, 118, 124, 125, 132, 134, 135, 138, 140, 141, 143, 144, 145, 146, 147, 148, 149, 151, 152, 153, 154, 160, 161, 163, 164, 165, 167, 169, 172, 177, 181, 188, 193, 195, 196, 198, 203, 204, 208, 210, 211, 215, 216, 221, 224, 226, 227, 228, 232, 236, 237, 239, 240, 250, 260, 261, 262, 263, 264, 265, 268, 276, 279, 280, 283, 286, 287, 292, 295, 297, 298, 304, 305, 306, 309, 310, 316, 319, 320, 321, 324, 332, 333, 334, 336, 337, 338, 339, 344, 345, 346, 348, 353, 358, 361, 364, 366, 369, 374, 379, 382, 384, 391, 393, 397, 401, 403, 405, 407, 412, 415, 416, 419, 422, 424, 425, 426, 429, 430, 432, 434, 435, 436, 437, 438, 440, 441, 443, 444, 446, 448, 449, 450]
labels = np.array([ 8,  6, 10,  1,  7, 14,  3, 16,  2,  4,  5,  9, 15])
thisdf = df.loc[indices]

print([(len(thisdf[thisdf[279]==label]) / 184)**2 for label in labels])
print(sum([(len(thisdf[thisdf[279]==label]) / 184)**2 for label in labels]))

In [28]:
thisdf

,0,1,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
75,34,0,165,65,88,159,371,167,92,88,...,-1.5,10.8,-1.4,0.0,0.0,0.4,2.2,15.1,30.5,10
56,38,1,160,63,79,0,376,165,0,34,...,0.0,10.1,0.0,0.0,0.0,0.0,1.5,26.2,37.0,1


In [ ]:
gini

In [ ]:
final_dt

In [ ]:
final_dt.root.feature

In [ ]:
final_dt.root.indices

In [ ]:
testcase = {9:25}
print(final_dt.predict(testcase))

In [ ]:
df

In [ ]:
labels = df[279].unique()
1 - sum((len(subdf[subdf[279]==label]) / total_rows) ** 2 for label in labels)

In [80]:
cols = [c for c in df.columns if df[c].dtype == 'int64']

y = df[cols][279]
X = df[cols].drop([279], axis=1)
# Split into training and test set 
X_train, X_test, y_train, y_test = train_test_split( 
             X, y, test_size = 0.2, random_state=42) 

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

# Calculate the accuracy of the model 
print("Accuracy:", clf.score(X_test, y_test)) 

Accuracy: 0.4725274725274725


In [81]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print('-'*55)
print('Confusion Matrix\n')
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.62      0.68      0.65        44
           2       0.14      0.29      0.19         7
           3       0.75      0.60      0.67         5
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         2
           6       0.33      0.33      0.33         6
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         1
           9       0.50      1.00      0.67         1
          10       0.71      0.31      0.43        16
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         6

    accuracy                           0.47        91
   macro avg       0.24      0.25      0.23        91
weighted avg       0.51      0.47      0.47        91

-------------------------------------------------------
Confusion Matrix

[[30 

/Users/joydeepbhattacharjee/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joydeepbhattacharjee/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [82]:
%%time

X_train[279] = y_train

decision_tree = DecisionTree.build_from(X_train)

/Users/joydeepbhattacharjee/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


CPU times: user 55 s, sys: 376 ms, total: 55.3 s
Wall time: 55.6 s


In [85]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,158,279
24,40,1,153,55,82,140,388,149,82,52,...,0,0,28,0,0,0,0,0,0,1
17,46,1,158,58,70,120,353,122,52,57,...,0,0,28,0,0,0,0,0,0,1
66,54,1,160,63,82,158,410,141,87,25,...,0,0,24,0,0,0,0,0,0,2
301,63,1,164,54,78,140,393,143,78,-1,...,0,0,24,0,0,0,0,0,0,1
356,47,0,175,94,101,175,336,229,118,-52,...,0,0,36,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,46,0,165,55,82,0,415,108,0,-1,...,0,0,36,0,0,0,0,0,0,6
270,72,1,160,70,79,152,368,156,97,77,...,0,0,28,0,0,0,0,0,0,1
348,34,1,165,60,74,178,302,127,117,112,...,0,0,24,0,0,0,0,0,0,5
435,39,1,164,62,79,155,367,153,95,50,...,0,0,28,0,0,0,0,0,0,1


In [84]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,149,150,151,152,153,154,155,156,157,158
299,51,0,170,82,90,155,382,216,88,9,...,44,0,0,28,0,0,0,0,0,0
39,45,0,175,80,94,163,401,159,106,-57,...,0,0,0,36,0,0,0,0,0,0
336,31,1,165,56,74,199,384,142,100,110,...,16,0,0,28,0,0,0,0,0,0
332,42,0,188,91,113,213,358,220,107,91,...,32,0,0,40,0,0,0,0,0,0
153,34,1,167,60,63,164,396,139,84,49,...,0,0,0,28,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,53,1,160,70,80,199,382,154,117,-37,...,52,0,0,24,0,0,0,0,0,0
137,47,0,162,74,102,126,357,135,83,55,...,48,0,0,24,0,0,0,0,0,0
93,62,1,165,70,72,169,328,135,85,-13,...,40,0,0,20,0,0,0,0,0,0
57,42,0,168,79,90,145,364,175,79,-11,...,44,0,0,24,0,0,0,0,0,0


In [92]:
y_pred = []
for index, row in X_test.iterrows():
    prediction = decision_tree.predict(row)
    y_pred.append(prediction)
    
print(classification_report(y_test, y_pred))
print('-'*55)
print('Confusion Matrix\n')
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.62      0.64      0.63        44
           2       0.17      0.29      0.21         7
           3       0.71      1.00      0.83         5
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         6
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         1
           9       0.25      1.00      0.40         1
          10       0.90      0.56      0.69        16
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         6

    accuracy                           0.49        91
   macro avg       0.20      0.27      0.21        91
weighted avg       0.51      0.49      0.49        91

-------------------------------------------------------
Confusion Matrix

[[28 

/Users/joydeepbhattacharjee/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joydeepbhattacharjee/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The outputs are very similar

in scikitlearn we are getting 47 f1 score

and in custom we are getting 49